In [6]:
import math
import os
import subprocess
import time
import scipy
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import pandas as pd
import itertools
import pickle
from tabulate import tabulate
from g4beam import *
from tqdm import *
from skopt import gp_minimize
import os

In [2]:

print(os.getcwd())  # Print current working directory
print(os.listdir())

/Applications/G4beamline-3.08.app/Contents/MacOS
['G4_PhaseRotation.g4bl', 'g4blgui', 'g4bl', 'g4bl-setup.csh', 'G4_RFCavity.g4bl', 'g4bldata', '__pycache__', 'g4beam.py', 'g4bl-setup.sh', 'g4blmpi', 'g4bltest', 'G4_FinalCooling_auto.g4bl', 'optimization.ipynb']


In [2]:
t_emit = 0.145  # mm
momentum = 100  # MeV/c
beta = 0.03    # m
alpha = 0.7     # dimensionless
l_emit = 1      # mm
pz_std = 1    # MeV/c
vd_dist = 24    # mm

pre_w1 = gen_distribution((beta, alpha, t_emit, 0, 0), (beta, alpha, t_emit, 0, 0), momentum, pz_std, z_emit=l_emit, N=20000)
pre_w1["PDGid"] = -13
print(pre_w1)
print_all_params(pre_w1)

              x         y  z         Px         Py          Pz         t  \
0     -1.982111 -1.950369  0   1.074455  14.527436   99.541046  0.245016   
1      1.239184  0.509172  0   4.106762   0.442134   99.332191  1.542012   
2      1.945728  3.346722  0  -1.377757   6.277505   98.511125 -0.195274   
3     -1.253275  2.856149  0  -5.268241  -5.381841   98.711529  0.646873   
4     -1.491126  0.168799  0   9.119859  -5.338006  100.979074 -0.502814   
...         ...       ... ..        ...        ...         ...       ...   
19995 -0.632299 -0.185436  0  14.802538   0.396076  101.727363 -0.024909   
19996 -0.519477  0.098044  0  -6.513484 -10.903833   96.949904  0.724136   
19997 -3.373118  5.625175  0   2.162149 -24.674827   95.870427 -2.287693   
19998  1.469223  1.541426  0 -15.836759  -2.712258   99.138512 -0.628894   
19999 -0.523544  0.898779  0   2.369654   1.103197   99.683810  1.393013   

       PDGid  EventID  TrackID  ...  ProperTime  PathLength PolX PolY PolZ  \
0        

In [4]:
# Function to optimize
def func(x):
    length, angle = x
    return emittances(cut_outliers(run_distribution(pre_w1, length, angle, vd_dist, axis=0)))[0]

start = time.time()
# Run optimization
optim_result = minimize(func, [7.5, 45], method="Nelder-Mead", bounds=((1, 10), (30, 70)), options=dict(fatol=1e-6))

# Get results
w1_length, w1_angle = optim_result.x
print(f"Length = {w1_length:.2f} mm\nAngle = {w1_angle:.1f} deg")
print("Time spent:", time.time()-start)

# Runs a single case with the optimal parameters
post_w1 = run_distribution(pre_w1, w1_length, w1_angle, vd_dist, axis=0)
print_all_params(post_w1)
print_all_params(cut_outliers(post_w1))

iter value
  36 4.11863e-02
Length = 6.96 mm
Angle = 46.8 deg
Time spent: 608.2452070713043
-----------------------------
Twiss parameters for X
emit  = 0.04414344402600207 mm
beta  = 0.035951219227144615 m
gamma = 193.5389900884439 1/m
alpha = -2.440893824333581
D     = 0.0205257924558626 m
D'    = -0.04911571011631675

Twiss parameters for Y
emit  = 0.15011674642211947 mm
beta  = 0.02545238415577693 m
gamma = 57.431895139937595 1/m
alpha = -0.6795429772251271
D     = 0.00021431655341890956 m
D'    = 0.014778416569957186

Z-emittance:  6.3167805243099515 mm
Z std: 143.1292816233489 mm
p std: 7.0717934754390175 MeV/c
Mean momentum: 88.1644093691325 MeV/c
-----------------------------
-----------------------------
Twiss parameters for X
emit  = 0.04118634672947639 mm
beta  = 0.03636122277702757 m
gamma = 199.78403402197287 1/m
alpha = -2.5028766985942816
D     = 0.02060860513216294 m
D'    = -0.05890123128134734

Twiss parameters for Y
emit  = 0.14761007978980617 mm
beta  = 0.0253577756

In [5]:
# Calculate dispersion correction
post_correct = remove_dispersion(post_w1)
print_all_params(post_correct)

# Ignore transverse momentums
no_transverse = remove_transverse(post_correct)

# Reverse transverse momentums in saved copy
reverse_transverse = post_correct.copy(deep=True)
reverse_transverse["Px"] *= -1
reverse_transverse["Py"] *= -1

-----------------------------
Twiss parameters for X
emit  = 0.044120759330828076 mm
beta  = 0.03596930704629806 m
gamma = 193.61302197769666 1/m
alpha = -2.4421560629241994
D     = -9.636562785294193e-06 m
D'    = -0.0015589353132289278

Twiss parameters for Y
emit  = 0.1501136365342665 mm
beta  = 0.025452638635622925 m
gamma = 57.43137385906625 1/m
alpha = -0.6795439685422674
D     = 2.5372455528655263e-06 m
D'    = 0.00035007456388579886

Z-emittance:  6.316717063590288 mm
Z std: 143.1285683576137 mm
p std: 7.0697895360768985 MeV/c
Mean momentum: 88.16346518498571 MeV/c
-----------------------------


In [3]:
drift_length = 16000
rf_freq = 0.025

start = time.time()
# Function to optimize
def func(x):
    rf_phase, rf_length, rf_grad = x
    drift_to_start = drift_length-rf_length/2
    post_drift = recenter_t(z_prop(no_transverse, drift_to_start))
    post_cavity = run_g4beam(post_drift, "G4_RFCavity.g4bl", RF_length=rf_length, frfcool=rf_freq, ficool=rf_phase, Vrfcool=rf_grad, nparticles=len(no_transverse))
    pre_w2 = recombine_transverse(post_cavity, reverse_transverse)
    return np.std(p_total(cut_pz(pre_w2)))

# Run optimization
optim_result = minimize(func, [0, 4700, 7], method="Nelder-Mead", bounds=((-90, 90), (2000, 6000), (1, 10)), options=dict(fatol=1e-6))

# Get results
rf_phase, rf_length, rf_grad = optim_result.x
print(f"Phase = {rf_phase:.2f} deg\nLength = {rf_length:.0f} mm\nGradient = {rf_grad:.2f} MV/m\nFrequency = {rf_freq*1000:.1f} MHz")
print("Time spent:", time.time()-start)

# Runs a single case with the optimal parameters and add the transverse back in
drift_to_start = drift_length-rf_length/2
post_drift = recenter_t(z_prop(no_transverse, drift_to_start))
post_cavity = cut_pz(recenter_t(run_g4beam(post_drift, "G4_RFCavity.g4bl", RF_length=rf_length, frfcool=rf_freq, ficool=rf_phase, Vrfcool=rf_grad)))
pre_w2 = recombine_transverse(post_cavity, reverse_transverse)
print_all_params(pre_w2)

iter value


NameError: name 'no_transverse' is not defined